# Nballs Generation

**Please read the [Readme](https://github.com/ghanem-mhd/N-Balls-Visualization) for more information.**

In [ ]:
from ipywidgets import Layout
import ipywidgets as widgets
import os, shutil, glob
from IPython.display import clear_output

SAMPLE = None

project_path        = os.path.dirname(os.getcwd())
data_path           = project_path + '/data'
custom_input        = data_path + "/custom_input.txt"
children_file_path  = data_path + "/small.children.txt"
cat_code_file_path  = data_path + "/small.catcode.txt"
word2vec            = data_path + "/glove.txt"
nballs_path         = data_path + "/nballs.txt"

def set_up_data_foler():
    if not os.path.isdir(data_path):
        os.mkdir(data_path)
    else:
        for file in glob.glob(data_path+"/*"):
            if not "glove" in file and not "data_out" in file:
                os.remove(file)
            elif "data_out" in file:
                shutil.rmtree(file)
        
def generate_files(glove_file_path, output_path, sample, input_file_path):
    generate_files_string_command = "../main.py --gen --w2v %s --output %s --sample %s --input %s"
    command = generate_files_string_command%(glove_file_path, output_path, sample, input_file_path)     
    %run $command
    check_input_for_nballs()
    
def generate_nballs(glove_file_path):
    generate_balls_string_command = "../main.py --train_nball %s --w2v %s --ws_child %s --ws_catcode %s --log %s/log.txt"
    command = generate_balls_string_command%(nballs_path, glove_file_path, children_file_path, cat_code_file_path, data_path)     
    %run $command
    if os.path.exists(nballs_path):
        print("Finihshed generating balls successfully.")
    else:
        print("Finihshed generating balls unsuccessfully.")
    
def write_data_to_file(file_path, lines):
    with open(file_path, 'w') as file:
        for line in lines:
            file.write(line)
            if not '\n' in line:
                file.write("\n")

# UI Widgets
input_examples         = widgets.Combobox( placeholder='Choose Example', description='Input:', options=['Cities','Fruits','Custom'], layout=Layout(height='auto', width='50%'))
check_input_button     = widgets.Button(description='Check', button_style='success')
words_input            = widgets.Textarea(value='',description='Words', placeholder='keywords comma separated',  layout=Layout(height='auto', width='50%'))
word2vec_path_input    = widgets.Text(placeholder='word2vec file path', value = '', description='word2vec', layout=Layout(height='auto', width='50%'))
generate_tree_button   = widgets.Button(description='Generate Tree', button_style='success')
generate_nballs_button = widgets.Button(description='Generate Balls', button_style='success')
buttons_box            = widgets.Box([word2vec_path_input,generate_tree_button])
custom_box             = widgets.Box([words_input,check_input_button])

# UI Widget functions
def on_input_example_choose(change):
    global SAMPLE
    if change['type'] == 'change' and change['name'] == 'value':
        choosen_values = change['new']
        input_examples.disabled = True
        SAMPLE = choosen_values
        if (choosen_values == 'Custom'):
            display(custom_box)
            SAMPLE = None
        else:
            display(buttons_box)

def on_check_button_clicked(button):
    parsed_tokens = [x.strip() for x in str(words_input.value).split(',')]
    words = [x for x in parsed_tokens if x]
    if len(words) > 0:
        words_input.disabled = True
        check_input_button.disabled = True
        write_data_to_file(custom_input,words)
        display(buttons_box)
    else:
        print("Can't parse your input! Try Again!")
        
def check_input_for_nballs():
    if os.path.exists(children_file_path) and os.path.exists(cat_code_file_path):
        display(generate_nballs_button)
        
def on_generate_tree_button_clicked(button):
    word2vec_path_input.disabled = True
    generate_tree_button.disabled = True
    if word2vec_path_input.value:
        generate_files(word2vec_path_input.value, data_path, SAMPLE, custom_input)
    else:
        print("Provide a path for word2Vec! Try again!")
        
        
def on_generate_balls_button_clicked(button):
    print("Start generating balls")
    generate_nballs_button.disabled = True
    generate_nballs(word2vec_path_input.value)
    
# Set Widget listeners
input_examples.observe(on_input_example_choose)
check_input_button.on_click(on_check_button_clicked)
generate_tree_button.on_click(on_generate_tree_button_clicked)
generate_nballs_button.on_click(on_generate_balls_button_clicked)

# Start UI nteraction
set_up_data_foler()
display(input_examples)